In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import clear_output          
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor

## Cargo el historial

In [2]:
users_ids_history = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/historial_cant_post_xuser_xarea.csv', dtype={i+1:'int8' for i in range(188)})
users_ids_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346764 entries, 0 to 346763
Columns: 189 entries, idpostulante to 187
dtypes: int8(188), object(1)
memory usage: 64.8+ MB


## Cargo set de datos

In [3]:
data_set_complete_v2 = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/data_set_complete_v2.csv', usecols=['sepostulo','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente'],\
                                dtype={'sepostulo':'int8', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8'})

data_set_complete_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11870235 entries, 0 to 11870234
Data columns (total 19 columns):
sepostulo           int8
idpostulante        object
veces_visto         int16
sexo                int8
edad                int8
titulo              object
nombre_area         category
estudio_encurso     int8
estudio_graduado    int8
tipo_full-time      int8
tipo_part-time      int8
tipo_otro           int8
zona_gba            int8
zona_caba           int8
nivel_senior        int8
nivel_junior        int8
nivel_otro          int8
nivel_jefe          int8
nivel_gerente       int8
dtypes: category(1), int16(1), int8(15), object(2)
memory usage: 396.2+ MB


Reduzco el set de datos

In [4]:
data = data_set_complete_v2.sample(1000000)

del data_set_complete_v2

data.reset_index(drop=True, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
sepostulo           1000000 non-null int8
idpostulante        1000000 non-null object
veces_visto         1000000 non-null int16
sexo                1000000 non-null int8
edad                1000000 non-null int8
titulo              1000000 non-null object
nombre_area         1000000 non-null category
estudio_encurso     1000000 non-null int8
estudio_graduado    1000000 non-null int8
tipo_full-time      1000000 non-null int8
tipo_part-time      1000000 non-null int8
tipo_otro           1000000 non-null int8
zona_gba            1000000 non-null int8
zona_caba           1000000 non-null int8
nivel_senior        1000000 non-null int8
nivel_junior        1000000 non-null int8
nivel_otro          1000000 non-null int8
nivel_jefe          1000000 non-null int8
nivel_gerente       1000000 non-null int8
dtypes: category(1), int16(1), int8(15), object(2)
memory usage: 33.4+ MB


Veamos los datos hasta este momento

In [5]:
data.head()

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,tipo_part-time,tipo_otro,zona_gba,zona_caba,nivel_senior,nivel_junior,nivel_otro,nivel_jefe,nivel_gerente
0,0,dY5lpw2,0,1,23,ADMINISTRATIVO JUNIOR DE INGENIERIA. IMPORTANT...,Administración,0,1,1,0,0,1,0,0,1,0,0,0
1,1,OqrzrjN,0,1,19,Merchandiser/Repositor - GBA Norte,Abastecimiento,0,1,1,0,0,1,0,1,0,0,0,0
2,1,GNZPVzv,2,1,28,Buscamos Vendedores para nuestras gondolas en ...,Ventas,0,1,1,0,0,1,0,0,0,1,0,0
3,0,1VLrvz,0,1,43,Gerente de Ventas - Máquinas viales.,Gerencia / Dirección General,4,0,1,0,0,1,0,1,0,0,0,0
4,0,ekmAdM9,0,1,26,Buscamos Vendedor de Góndola,Ventas,0,4,1,0,0,1,0,0,0,1,0,0


## Cargo el test100k

In [6]:
test100k = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/test100k_complete_v2.csv', usecols=['id','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente'],\
                                dtype={'id':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8'})

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
id                  100000 non-null int64
idpostulante        100000 non-null object
veces_visto         100000 non-null int16
sexo                100000 non-null int8
edad                100000 non-null int8
titulo              100000 non-null object
nombre_area         100000 non-null category
estudio_encurso     100000 non-null int8
estudio_graduado    100000 non-null int8
tipo_full-time      100000 non-null int8
tipo_part-time      100000 non-null int8
tipo_otro           100000 non-null int8
zona_gba            100000 non-null int8
zona_caba           100000 non-null int8
nivel_senior        100000 non-null int8
nivel_junior        100000 non-null int8
nivel_otro          100000 non-null int8
nivel_jefe          100000 non-null int8
nivel_gerente       100000 non-null int8
dtypes: category(1), int16(1), int64(1), int8(14), object(2)
memory usage: 4.0+ MB


Chequeo que el test tenga misma estructura

In [7]:
test100k.head()

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,tipo_part-time,tipo_otro,zona_gba,zona_caba,nivel_senior,nivel_junior,nivel_otro,nivel_jefe,nivel_gerente
0,0,6M9ZQR,0,0,42,Asistente Comercial,Comercial,0,7,1,0,0,0,1,0,0,0,1,0
1,1,6v1xdL,0,1,30,Asistente Comercial,Comercial,2,3,1,0,0,0,1,0,0,0,1,0
2,2,ezRKm9,0,0,36,Asistente Comercial,Comercial,0,4,1,0,0,0,1,0,0,0,1,0
3,3,1Q35ej,0,1,68,Oracle Financials Sr. Application Developer,Tecnologia / Sistemas,0,5,1,0,0,0,1,0,0,1,0,0
4,4,EAN4J6,0,0,32,Oracle Financials Sr. Application Developer,Tecnologia / Sistemas,4,0,1,0,0,0,1,0,0,1,0,0


## Taking care of nombre_area

#### Primero encoders para eso

In [8]:
avisos_totales = pd.concat([pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv', usecols=['idaviso', 'nombre_area'])])
avisos_totales.drop_duplicates(subset=['idaviso'], inplace=True)
avisos_totales.reset_index(drop=True, inplace=True)
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 2 columns):
idaviso        25288 non-null int64
nombre_area    25288 non-null object
dtypes: int64(1), object(1)
memory usage: 395.2+ KB


In [9]:
avisos_totales['nombre_area'].value_counts().size

188

In [10]:
le = preprocessing.LabelEncoder()
avisos_totales['codigo_area'] = le.fit_transform(avisos_totales['nombre_area'].astype(str))

#PARA EL ONE HOT ENCODING, AVISARLE QUE LA CANTIDAD DE VALORES CATEGORICOS ES EL DE TODAS LAS CATEGORIAS EN EL DF DE TODOS LOS ANUNCIOS:188
enc = preprocessing.OneHotEncoder(188)   
enc.fit(avisos_totales['codigo_area'].values.reshape(-1,1))

avisos_totales.head()

,idaviso,nombre_area,codigo_area
0,8725750,Comercial,30
1,17903700,Salud,158
2,1000150677,Transporte,181
3,1000610287,Transporte,181
4,1000872556,Producción,143


In [11]:
data['nombre_area'] = le.transform(data['nombre_area'].astype(str))
test100k['nombre_area'] = le.transform(test100k['nombre_area'].astype(str))

In [12]:
encoded_areas = enc.transform(data['nombre_area'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

data = data.merge(encoded_areas_df, left_index=True, right_index=True)

del encoded_areas, encoded_areas_df

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 207 entries, sepostulo to 187
dtypes: int16(1), int64(1), int8(203), object(2)
memory usage: 218.4+ MB


In [13]:
encoded_areas = enc.transform(test100k['nombre_area'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

test100k = test100k.merge(encoded_areas_df, left_index=True, right_index=True)

del encoded_areas, encoded_areas_df

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 207 entries, id to 187
dtypes: int16(1), int64(2), int8(202), object(2)
memory usage: 22.5+ MB


Veamos como quedo todo despues de los encoders

In [14]:
data.head()

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,dY5lpw2,0,1,23,ADMINISTRATIVO JUNIOR DE INGENIERIA. IMPORTANT...,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,OqrzrjN,0,1,19,Merchandiser/Repositor - GBA Norte,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,GNZPVzv,2,1,28,Buscamos Vendedores para nuestras gondolas en ...,185,0,1,1,...,0,0,0,0,0,0,0,1,0,0
3,0,1VLrvz,0,1,43,Gerente de Ventas - Máquinas viales.,83,4,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,ekmAdM9,0,1,26,Buscamos Vendedor de Góndola,185,0,4,1,...,0,0,0,0,0,0,0,1,0,0


In [15]:
test100k.head()

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,0,0,0
2,2,ezRKm9,0,0,36,Asistente Comercial,30,0,4,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1Q35ej,0,1,68,Oracle Financials Sr. Application Developer,170,0,5,1,...,0,0,0,0,0,0,0,0,0,0
4,4,EAN4J6,0,0,32,Oracle Financials Sr. Application Developer,170,4,0,1,...,0,0,0,0,0,0,0,0,0,0


## Mergeo con el historial

In [16]:
data = data.merge((data['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int8', copy=False)), left_index=True, right_index=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 395 entries, sepostulo to 187
dtypes: int16(1), int64(1), int8(391), object(2)
memory usage: 397.7+ MB


In [17]:
test100k = test100k.merge((test100k['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int8', copy=False)), left_index=True, right_index=True)

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 395 entries, id to 187
dtypes: int16(1), int64(2), int8(390), object(2)
memory usage: 40.4+ MB


### Estos serian nuestros set de datos y test finales

In [18]:
data.head()

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,dY5lpw2,0,1,23,ADMINISTRATIVO JUNIOR DE INGENIERIA. IMPORTANT...,2,0,1,1,...,0,0,0,0,0,0,0,2,0,0
1,1,OqrzrjN,0,1,19,Merchandiser/Repositor - GBA Norte,0,0,1,1,...,0,0,0,0,0,0,0,3,0,0
2,1,GNZPVzv,2,1,28,Buscamos Vendedores para nuestras gondolas en ...,185,0,1,1,...,0,0,0,2,0,0,0,4,0,0
3,0,1VLrvz,0,1,43,Gerente de Ventas - Máquinas viales.,83,4,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,ekmAdM9,0,1,26,Buscamos Vendedor de Góndola,185,0,4,1,...,0,0,0,0,0,0,0,0,0,0


In [19]:
test100k.head()

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,4,0,0
2,2,ezRKm9,0,0,36,Asistente Comercial,30,0,4,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1Q35ej,0,1,68,Oracle Financials Sr. Application Developer,170,0,5,1,...,0,0,0,0,0,0,0,0,0,0
4,4,EAN4J6,0,0,32,Oracle Financials Sr. Application Developer,170,4,0,1,...,0,0,0,0,0,0,0,0,0,0


## Genero los sets de entrenamiento 

In [20]:
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['edad','sepostulo','idpostulante', 'titulo']), data['sepostulo'], test_size=0.3)

In [21]:
 bagging = BaggingClassifier(oob_score= True)

In [22]:
bagging.fit(train_set, train_classes)

/home/rodrigo/ENV/lib/python3.5/site-packages/sklearn/ensemble/bagging.py:605: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/rodrigo/ENV/lib/python3.5/site-packages/sklearn/ensemble/bagging.py:610: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=True, random_state=None,
         verbose=0, warm_start=False)

In [23]:
bagging.score(test_set, test_classes)

0.89508

In [24]:
test100k['sepostulo'] = bagging.predict(test100k.drop(columns=['edad','id','idpostulante', 'titulo']))

In [25]:
test100k['sepostulo'].value_counts()

0    54913
1    45087
Name: sepostulo, dtype: int64

In [26]:
test100k[['id','sepostulo']].to_csv('bagging_data_v2_usando_decision_tree.csv', index=False)

In [28]:
del test100k['sepostulo']

## Voy a probar usando random forest

In [27]:
rf = RandomForestClassifier(n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)

In [29]:
bagging_con_rf = BaggingClassifier(rf, oob_score= True)

In [30]:
bagging_con_rf.fit(train_set, train_classes)

/home/rodrigo/ENV/lib/python3.5/site-packages/sklearn/ensemble/bagging.py:605: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/rodrigo/ENV/lib/python3.5/site-packages/sklearn/ensemble/bagging.py:610: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=True,
         random_state=None, verbose=0, warm_start=False)

In [31]:
bagging_con_rf.score(test_set, test_classes)

0.8830566666666667

In [33]:
test100k['sepostulo'] = bagging_con_rf.predict(test100k.drop(columns=['edad','id','idpostulante', 'titulo']))

In [34]:
test100k['sepostulo'].value_counts()

1    50186
0    49814
Name: sepostulo, dtype: int64

In [35]:
test100k[['id','sepostulo']].to_csv('bagging_data_v2_usando_random_forest.csv', index=False)

In [36]:
del test100k['sepostulo']

## Uso logistic regression

In [21]:
regresion = GradientBoostingRegressor()

In [22]:
bagging_con_regresion = BaggingClassifier(regresion, oob_score= True)

In [23]:
bagging_con_regresion.fit(train_set, train_classes)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices